In [1]:
import psycopg2

# create db connnection
def create_connection():
    try:
        connnection = psycopg2.connect(
            dbname="musicstreamingdb",
            user="postgres",
            password="postgrespass",
            host="localhost"
        )
        return connnection
    except Exception as e:
        print('Error connecting to the db:', e)
        return None

    connection.create_connection()
    cursor = connection.cursor()

In [2]:
# Function to execute SQL query and print the result
def execute_sql(query, fetch_result=False):
    connection = create_connection()
    if connection:
        try:
            cursor = connection.cursor()
            cursor.execute(query)
            
            if fetch_result:
                result = cursor.fetchall()  # Fetch all results for SELECT queries
                return result  # Return result to be printed by the caller
            
            connection.commit()  # Commit changes 
            print("Query executed successfully.")
        except Exception as e:
            print("Error executing SQL query:", e)
        finally:
            cursor.close()
            connection.close()

# Function to run a query and print results for SELECT queries
def execute_and_print_query(query):
    result = execute_sql(query, fetch_result=True)
    if result:
        for row in result:
            print(row)
    else:
        print("No data returned.")

In [ ]:
'''Basic select with simple where clause.'''
QUERY0 = """
SELECT * 
FROM Songs 
WHERE genre = 'Pop'
"""

'''Basic select with simple group by clause (with and without having clause).'''
QUERY01 = """
SELECT artist, COUNT(*) 
FROM Songs 
GROUP BY artist
"""
QUERY02 = """
SELECT artist, COUNT(*) 
FROM Songs GROUP BY artist 
HAVING COUNT(*) > 5; 
"""

'''A simple join query as well as its equivalent implementation using cartesian product and where clause.'''
QUERY03 = """
SELECT s.title, sp.spotify_url 
FROM Songs s 
JOIN SpotifySongs sp ON s.song_id = sp.song_id;
"""

QUERY04 = """
SELECT s.title, sp.spotify_url 
FROM Songs s, SpotifySongs sp 
WHERE s.song_id = sp.song_id;
"""
execute_and_print_query(QUERY0)
execute_and_print_query(QUERY01)
execute_and_print_query(QUERY02)
execute_and_print_query(QUERY03)
execute_and_print_query(QUERY04)


In [ ]:
'''
    #5 An example of a query that returns songs that have a null field(s)
'''
# This query displays songs that have a null genre
QUERY1 = """
SELECT * 
FROM Songs
WHERE genre IS NULL OR release_date IS NULL;
"""

# This query displays songs that have both genre and release_date fields as null
QUERY2 = """
SELECT * 
FROM Songs
WHERE genre IS NULL OR release_date IS NULL;
"""
execute_and_print_query(QUERY1)
execute_and_print_query(QUERY2)

In [1]:
'''
    #8 An example of a view that has a hard-coded criteria, by which the content of the view may change upon changing the hard-coded value
'''
# This query will display the spotify song's with a popularity score above 80
QUERY = """
CREATE OR REPLACE VIEW HighPopularitySpotifySongs AS 
SELECT S.title, S.artist, S.genre, SS.spotify_popularity_score AS popularity_scr
FROM Songs S
JOIN SpotifySongs SS ON S.song_id = SS.song_id
WHERE SS.spotify_popularity_score > 80;
SELECT * FROM HighPopularitySpotifySongs;
"""
execute_and_print_query(QUERY)

NameError: name 'execute_and_print_query' is not defined

In [ ]:
'''
    #9 Two queries that demonstrate the overlap and covering constraints.
'''
# Overlap:
QUERY1 = """
SELECT S.title FROM Songs S
JOIN SpotifySongs SS ON S.song_id = SS.song_id
JOIN LastfmSongs LFMS ON S.song_id = LFMS.song_id;
"""

# Covering:
QUERY2= """ 
SELECT S.title 
FROM Songs S 
WHERE NOT EXISTS (
    SELECT 1 FROM SpotifySongs AS SS 
    WHERE SS.song_id = S.song_id
)
AND NOT EXISTS (
    SELECT 1 FROM LastfmSongs AS LFMS 
    WHERE LFMS.song_id = S.song_id
);
"""
execute_and_print_query(QUERY1)
execute_and_print_query(QUERY2)

No data returned.
No data returned.


In [6]:
'''
    #10 Two implementations of the division operator using 
         a) a regular nested query using NOT IN 
         b) a correlated nested query using NOT EXISTS and EXCEPT 
'''
# Both questies ensure that a song exists in both Songs and SpotifySongs 
# Using NOT IN:
QUERY1 = """
SELECT S.title 
FROM Songs S
WHERE S.song_id IN (
    SELECT SS.song_id FROM SpotifySongs SS
);
"""

# Using NOT EXISTS and EXCEPT:
QUERY2 = """ 
SELECT S.title 
FROM Songs S
WHERE NOT EXISTS (
    SELECT S.song_id
    EXCEPT
    SELECT SS.song_id
    FROM SpotifySongs SS
);
"""
execute_and_print_query(QUERY1)
execute_and_print_query(QUERY2)

('Die With A Smile',)
('APT.',)
('BIRDS OF A FEATHER',)
('That’s So True',)
('Who',)
('Sailor Song',)
('Taste',)
('Tu Boda',)
('WILDFLOWER',)
('Running Wild',)
('Espresso',)
('Timeless (with Playboi Carti)',)
('Good Luck, Babe!',)
('The Emptiness Machine',)
('Si Antes Te Hubiera Conocido',)
('Beautiful Things',)
("I Love You, I'm Sorry",)
('Please Please Please',)
('The Night We Met',)
('All I Want for Christmas Is You',)
('Die With A Smile',)
('APT.',)
('BIRDS OF A FEATHER',)
('That’s So True',)
('Who',)
('Sailor Song',)
('Taste',)
('Tu Boda',)
('WILDFLOWER',)
('Running Wild',)
('Espresso',)
('Timeless (with Playboi Carti)',)
('Good Luck, Babe!',)
('The Emptiness Machine',)
('Si Antes Te Hubiera Conocido',)
('Beautiful Things',)
("I Love You, I'm Sorry",)
('Please Please Please',)
('The Night We Met',)
('All I Want for Christmas Is You',)
